# Julia methods as actors

**Requirements:** PyCall, pyjulia

In [1]:
%load_ext autoreload
%autoreload(2)

In [2]:
from wowp.actors.julia import JuliaMethod
from wowp.schedulers import LinearizedScheduler
import numpy as np

## Simple calling

In [3]:
sqrt = JuliaMethod("sqrt", inports="a")
sqrt(4)

2.0

## Calling on numpy arrays

In [4]:
sqrt = JuliaMethod("sqrt", inports="a")

array = np.random.rand(5, 5)
scheduler = LinearizedScheduler()
scheduler.put_value(sqrt.inports.a, array)
scheduler.execute()
sqrt.outports.result.pop()

array([[ 0.81698787,  0.62958974,  0.88896111,  0.94228041,  0.7122345 ],
       [ 0.7487566 ,  0.89026623,  0.6646635 ,  0.95888315,  0.79889142],
       [ 0.81541781,  0.72291517,  0.98658867,  0.92325251,  0.72675485],
       [ 0.38424982,  0.94999526,  0.47267868,  0.35454016,  0.16124045],
       [ 0.82816266,  0.75291426,  0.67140565,  0.8741716 ,  0.80376571]])

## Chain sqrt method to pass numpy arrays

In [5]:
sqrt = JuliaMethod("sqrt", inports="a")
sqrt2 = JuliaMethod("sqrt", inports="a")

sqrt.outports.result.connect(sqrt2.inports.a)

array = np.random.rand(5, 5)
scheduler = LinearizedScheduler()
scheduler.put_value(sqrt.inports.a, array)
scheduler.execute()
sqrt2.outports.result.pop()

array([[ 0.89297378,  0.98111455,  0.93098508,  0.99167704,  0.9145368 ],
       [ 0.99198595,  0.90823154,  0.91498147,  0.89229372,  0.79410293],
       [ 0.93072651,  0.73233775,  0.61051038,  0.89162012,  0.98378698],
       [ 0.99416325,  0.8326235 ,  0.92652858,  0.84859751,  0.67992983],
       [ 0.98926214,  0.97013604,  0.59582316,  0.42559714,  0.99660929]])

## Using method from a package

In [15]:
%%file ABCD.jl

module ABCD

VERSION < v"0.4-" && using Docile

export quad

@doc doc"""Fourth power of the argument.""" ->
function quad(a)
    a ^ 4
end

end

Overwriting ABCD.jl


In [16]:
quad = JuliaMethod(package_name="ABCD", method_name="quad", inports="a")
quad(4.0)

256.0

In [17]:
quad.name

'ABCD.quad'

### Non-existent module

In [20]:
xxx = JuliaMethod(package_name="ABCD", method_name="x")
xxx()

JuliaError: Exception 'UndefVarError' ocurred while calling julia code:
 Ny

Code:
ABCD.x

In [19]:
xxx = JuliaMethod(package_name="ABCD", method_name="x")
xxx()

JuliaError: Exception 'UndefVarError' ocurred while calling julia code:
 Ny

Code:
ABCD.x